<a href="https://colab.research.google.com/github/sophiexingsu/HCPpipelines/blob/master/ML_dyslexia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import packages and the dataset

In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
data =pd.read_csv("/content/machine_sampled.csv") 
y=pd.read_csv("/content/machine_sample_Ys.csv") 


### Explore the dataset, normalize and standarize the dataset

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
norm = MinMaxScaler()
standard = StandardScaler()

In [ ]:
area=data.iloc[:,1:365]
norm = MinMaxScaler().fit(area)
scaler = preprocessing.StandardScaler()
names = area.columns
area_norm = norm.transform(area)
scaled_area=scaler.fit_transform(area_norm)
word=y.iloc[:,1]
pseudo=y.iloc[:,2]
comp=(word+pseudo)/2 
print(word)

0      21
1      65
2      42
3      25
4      75
       ..
788    57
789    52
790    23
791    19
792    64
Name: wiat_word_r, Length: 793, dtype: int64


In [ ]:
ct=data.iloc[:,366:727]
norm = MinMaxScaler().fit(ct)
scaler = preprocessing.StandardScaler()
names = ct.columns
ct_norm = norm.transform(ct)
scaled_ct=scaler.fit_transform(ct_norm)

In [ ]:
volume=data.iloc[:,728:1089]
norm = MinMaxScaler().fit(volume)
scaler = preprocessing.StandardScaler()
names = volume.columns
volume_norm = norm.transform(volume)
scaled_volume=scaler.fit_transform(volume_norm)

In [ ]:
meancur=data.iloc[:,1089:1451]
norm = MinMaxScaler().fit(meancur)
scaler = preprocessing.StandardScaler()
names = meancur.columns
meancur_norm = norm.transform(meancur)
scaled_meancur=scaler.fit_transform(meancur_norm)


In [ ]:
fold=data.iloc[:,1451:1813]
norm = MinMaxScaler().fit(fold)
scaler = preprocessing.StandardScaler()
names = fold.columns
fold_norm = norm.transform(fold)
scaled_fold=scaler.fit_transform(fold_norm)
combined_parameter=np.hstack((area,ct,fold,meancur,volume))
combined_parameter_stand=np.hstack((scaled_area,scaled_ct,scaled_meancur,scaled_volume,scaled_fold))

In [ ]:
gender=data.iloc[:,1819:1819]
site=data.iloc[:,1815:1815] 
age=data.iloc[:,1816:1816]
all_para_stand=np.hstack((combined_parameter_stand,gender,site,age))


In [ ]:
#try to regress out some confounding varibales before do things 
import statsmodels.api as sm
y = combined_parameter_stand
X = np.hstack((gender,site,age))
X = sm.add_constant(X) 
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
#ypred.shape
parameter_regressed_stand=y-ypred

In [ ]:
y=combined_parameter_stand
X = np.hstack((gender,site,age))
X = sm.add_constant(X) 
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
#ypred.shape
parameter_regressed_stand=y-ypred

In [ ]:
comp=(word+pseudo)/2 
y=comp
X = np.hstack((gender,site,age))
X = sm.add_constant(X) 
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
comp_regressed=y-ypred

In [ ]:
###combined_parameter_read
combined_as_dataset=pd.read_csv("/content/combined_parameter_stand.csv") 


#### features selection proceeses 

In [ ]:

from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest, chi2
#from sklearn.feature_selection import chi2
#apply SelectKBest class to extract top 10 best features
X=parameter_regressed_stand
y=comp_regressed
#bestfeatures = SelectKBest(score_func=chi2, k=100)
#fit = bestfeatures.fit(X,y)
#dfscores = pd.DataFrame(fit.scores_)
#dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
#featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#featureScores.columns = ['Specs','Score']  #naming the dataframe columns
#print(featureScores.nlargest(10,'Score'))  #print 10 best features

X_new=SelectKBest(chi2, k=500).fit_transform(X, y)

print(X.shape)

(793, 1810)


Trying: how to do the proces with CV 

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn import datasets, linear_model
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X=parameter_regressed_stand
y=comp_regressed
fs = SelectKBest(score_func=f_regression, k=100)
model = linear_model.Lasso()
pipeline = Pipeline(steps=[('s',fs),('m',model)])
# evaluate model
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=5, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: -0.018 (0.014)


multivariable comparsion

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, word_regressed, test_size = 0.2,random_state = 102)
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))

-1.5998387695064498


In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from numpy import absolute
from numpy import mean
from numpy import std
model = linear_model.Lasso()
scores= cross_val_score(model, combined_parameter_stand,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
#scores = cross_val_score(model, X, word, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.551 (1.061)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, parameter_regressed_norm,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.345 (1.088)


In [ ]:
model = linear_model.Lasso()

In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, parameter_regressed_norm,word, cv=10,scoring='r2')
scores = absolute(scores)
print('R^2: %.3f (%.3f)' % (mean(scores), std(scores)))

NameError: ignored

In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, parameter_regressed_stand,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.551 (1.061)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, combined_parameter_stand,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.149 (1.060)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, parameter_regressed_norm,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.123 (1.048)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, combined_parameter_stand,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.498 (1.142)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, parameter_regressed_norm,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.513 (1.190)


In [ ]:
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest, f_classif
X=parameter_regressed_norm
y=word
para50_new=SelectKBest(score_func=f_classif, k=50).fit_transform(X, y)
para100_new=SelectKBest(score_func=f_classif, k=100).fit_transform(X, y)
para300_new=SelectKBest(score_func=f_classif, k=300).fit_transform(X, y)
para500_new=SelectKBest(score_func=f_classif, k=500).fit_transform(X, y)
para1000_new=SelectKBest(score_func=f_classif, k=1000).fit_transform(X, y)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 50)
fit = rfe.fit(X, y)
para50=fit.transform(X)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, para50_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.996 (1.348)


In [ ]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, para100_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 17.767 (1.015)


In [ ]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, para300_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.963 (1.200)


In [ ]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, para500_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.537 (1.362)


In [ ]:
model = RandomForestRegressor(n_estimators = 10, random_state = 0)
scores= cross_val_score(model, para1000_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.655 (1.050)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, para1000_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.104 (1.045)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, para1000_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.345 (1.088)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, para500_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.106 (1.006)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, para300_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.142 (1.002)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, para100_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.198 (1.079)


In [ ]:
model = SVR(kernel = 'rbf')
scores= cross_val_score(model, para50_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.222 (1.101)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, para50_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))


MAE: 16.345 (1.088)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, para100_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.345 (1.088)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, para300_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.345 (1.088)


In [ ]:
model = linear_model.Lasso()
scores= cross_val_score(model, para500_new,word, cv=10,scoring='neg_mean_absolute_error')
scores = absolute(scores)
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 16.345 (1.088)


In [ ]:
from sklearn.svm import SVR
X_train, X_test, y_train, y_test = train_test_split(X, word_regressed, test_size = 0.2, random_state = 102)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

-0.016280067104220475


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(parameter_regressed_norm, comp, test_size = 0.2, random_state = 0)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

0.00963397509660624


###Multivariable Linear Regression for all the parameters

####linear regression with norms, word

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(parameter_regressed_stand, word_regressed, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=parameter_regressed_stand
y=word_regressed
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-1.9087989613734764
[-5.46772074e+02 -6.83358223e+01 -2.23865795e+00 -1.78640000e+03
 -3.88494017e+00 -1.50824787e+00 -5.22858691e+00 -4.33793105e+02
 -8.57867318e+01 -2.37282907e+03]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(parameter_regressed_norm, word_regressed, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=parameter_regressed_norm
y=word_regressed
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-1.9085662178105878
[-5.53862048e+02 -7.01233408e+01 -2.23793159e+00 -1.74022803e+03
 -3.88402932e+00 -1.50775974e+00 -5.22887897e+00 -2.65300259e+02
 -8.71661895e+01 -2.98132705e+03]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(combined_parameter_norm, word_regressed, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=combined_parameter_norm
y=word_regressed
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-1.868922818095395
[-5.60233231e+02 -7.17027786e+01 -2.23896856e+00 -1.87844754e+03
 -3.90576129e+00 -1.50688064e+00 -4.97466427e+00 -3.79859435e+02
 -1.02578116e+02 -2.97698867e+03]


#### liner regression with norm, comp

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_norm, comp, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
#try to normalize the dataset 
from sklearn.preprocessing import MinMaxScaler
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=all_para_norm
y=comp
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-2.1644898803144637
[-4.34038084e+02 -6.30982854e+01 -2.64879420e+00 -1.86535797e+03
 -3.78788417e+00 -1.57320393e+00 -5.13886965e+00 -3.73567129e+02
 -5.71581399e+01 -2.41028955e+03]


#### linear regression with norm, pseudo, and the take way was that word was a very good predictors

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_norm, pseudo, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
#try to normalize the dataset 
from sklearn.preprocessing import MinMaxScaler
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=all_para_norm
y=pseudo
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-2.7456640522887628
[ -260.60539097   -41.88227694    -3.46642881 -1632.86080024
    -3.39504418    -1.67008534    -4.90814085  -328.11395391
   -15.37460912 -1544.75396295]


#### repeat the procedure but wiht standardized version

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_stand, word, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
#try to normalize the dataset 
from sklearn.preprocessing import MinMaxScaler
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
X=all_para_stand
y=word
scores= cross_val_score(clf, X,y, cv=10)
print(scores)

-1.9130777826782412
[-5.39628125e+02 -6.98635912e+01 -2.23941775e+00 -1.79042526e+03
 -3.88492429e+00 -1.50769183e+00 -5.22468128e+00 -4.47578367e+02
 -8.57892618e+01 -2.30965362e+03]


###Multivariable Linear Regression with PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_stand, word, test_size = 0.2,random_state= 0)
regressor = svm.SVR()
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))
#regressor.coef_
#need_to_think_about_and_try_again_with_PCA!!!!! 

-0.007734546654104779


###svm regression with all parameters with cross validations



In [ ]:
from sklearn.svm import SVR
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(all_para_stand, word, test_size = 0.2, random_state = 0)
clf = svm.SVC(kernel='rbf')
#regressor = SVR(kernel = 'rbf')
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
X=all_para_stand
y=word
print(r2_score(y_test,y_pred))
#scores= cross_val_score(clf, X,y, cv=10)
#print(scores)


-1.093102015444881


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_norm, word, test_size = 0.2)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

0.019209207462398514


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_norm, pseudo, test_size = 0.2,random_state= 0)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

0.01398966885200248


In [ ]:
from sklearn.svm import SVR
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(ct, wiat, test_size = 0.2, random_state = 0)
regressor = svm.SVR()
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

In [ ]:
from sklearn.svm import SVR
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(ct, wiat, test_size = 0.2, random_state = 0)
regressor = svm.SVR()
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_para_stand, word, test_size = 0.2)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

-0.08348306143033546


##Try feature selections to imporve R2 model-fitting results

####1. Univariate Selection(only think about select numbers of featuers)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from numpy import set_printoptions
test = SelectKBest(score_func=f_classif, k=50)
X=all_para_stand
y=word
fit = test.fit(X, y)
set_printoptions(precision=3)
#print(fit.scores_)
features = fit.transform(X)
#print(features[0:,:])
X_train, X_test, y_train, y_test = train_test_split(features, word, test_size = 0.2, random_state = 0)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
#print(r2_score(y_test,y_pred))
print(features.shape)

(793, 50)


#### 2.Recursive Feature Elimination

### Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
X_train, X_test, y_train, y_test = train_test_split(every_parameters, comp, test_size = 0.2, random_state = 0)
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

-0.07933995472108046


##Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(area, comp, test_size = 0.2, random_state = 0)
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

-0.13007939087759213


###SVM regression with PCA

##Multivariable Linear Regression(with only one types of information)


In [ ]:
#X=data.iloc[:,2:369].values
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X = sc.fit_transform(X)
#y=data.iloc[:,-2].values
X_train, X_test, y_train, y_test = train_test_split(combined_parameter, y, test_size = 0.2, random_state = 0)
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(r2_score(y_test,y_pred))
scores = cross_val_score(clf, X,y, cv=10)
print(scores)

-4.089212457299408
[-0.97887247 -1.28835842 -1.44422008 -1.45670776 -1.06571198 -1.40981729
 -0.69923256 -0.91888026 -1.59579653 -1.30369002]


### Support Vector Machine Regression 

In [ ]:
from sklearn.svm import SVR
X_train, X_test, y_train, y_test = train_test_split(every_parameters,comp, test_size = 0.3, random_state = 0)
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))



0.028911951087734034


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, y], axis = 1)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
colors = ['r', 'g', 'b']
for y, color in zip(,colors):
    indicesToKeep = finalDf[y] == y
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.grid()

SyntaxError: ignored

In [ ]:
X.shape

(793, 98)

In [ ]:
y=data.iloc[:,-2].values

In [ ]:
y.shape

(793,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
scores = cross_val_score(clf, X,y, cv=10)
print(scores)
print(clf.coef_)

[-0.09012661 -0.30647211 -0.39852271 -0.02823317 -0.19139026 -0.30580415
 -0.21873811 -0.08901324 -0.19350537 -0.24796874]
[ 1.53200944  6.14504377 -0.73043865  1.56824801 -0.04528997 -2.8665912
 -4.06170411  1.43058957  2.11021784  3.79127231 -1.79876737 -0.36451589
  3.35952943 -0.98410856 -3.74676844 -0.35590461  1.85223914 -0.94307277
  0.81671712  1.04293699 -1.86377718 -0.59585927 -2.82010897 -1.43362552
  0.01456814 -0.24281174 -1.6326389  -3.15518718 -0.99720899 -0.86129899
 -1.63190801  0.61555054  4.15744102  1.11807002  2.0667648  -0.69803976
 -0.21153292  2.52088038 -1.78022035  0.74936501 -0.70379107 -3.76003576
 -3.40760843 -2.56554056 -1.15881968  3.98281968  0.67405438  0.81588501
 -3.69983617  2.85263329 -1.62069308 -1.01856309  0.75336553 -2.77420788
  1.96532093  1.00148832 -3.56594243 -1.18451754  0.01978384  1.61000901
  1.21403289  0.55524566 -0.38454257  4.50956713 -2.25475188  0.92145453
 -0.05177841  2.02658234  0.38929675  4.70498993 -0.09877979 -1.44811286
 -

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred))

0.01748111361124971


In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_pred = regressor.predict(X_test)
scores = cross_val_score(regressor, X,y, cv=10)

In [ ]:
print(scores)

[-0.01147751  0.01434016 -0.03609378 -0.01820892  0.03018026 -0.00665072
 -0.0861158   0.01831833 -0.08197282  0.03434223]


In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X, y)
scores = cross_val_score(regressor, X,y, cv=10)
print(scores)

[-0.99257612 -0.89847647 -0.87575298 -0.99887658 -1.14617417 -0.85358699
 -1.26657274 -0.5386604  -1.04505316 -1.34881147]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X, y)
scores = cross_val_score(regressor, X,y, cv=10)
print(scores)

[ 0.01823505 -0.1632563  -0.14407564 -0.10535155 -0.14579217 -0.08041626
 -0.28837182 -0.04968458 -0.20078447 -0.09528899]


In [ ]:
##learns how to do feature selection in regression problems

NameError: ignored